[View in Colaboratory](https://colab.research.google.com/github/mari-linhares/tensorflow-brasil/blob/eager-hub/tfhub_classificador_caes_e_gatos.ipynb)

## Dependências

In [3]:
!pip install tensorflow-hub
!pip install pillow matplotlib numpy pandas

#### Importando dependências

In [1]:
# Compatibilidade entre Python 2 e Python 3
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import sys
import os
try:
  import urllib.request as urrlib_request # Python 3
except:
  import urllib as urllib_request  # Python 2
import tarfile
import zipfile


# TensorFlow
import tensorflow as tf
import tensorflow_hub as hub
tf.logging.set_verbosity(tf.logging.INFO)  # Permitindo visualização de logs

# Bibliotecas auxiliares
try:
  import cPickle  # maior eficiência ao processar as imagens
except:
  import _pickle as cPickle

import numpy as np  # manipular vetores
from PIL import Image  # lidar com imagens
import matplotlib.pyplot as plt  # plotar imagens
%matplotlib inline

import glob  # lidar com arquivos

# IMPORTANTE: essa linha garante que os números gerados aleatoriamente são previsíveis
np.random.seed(0)


tf.logging.info('Recomenda-se para esta atividade uma versão >= 1.7.0')
tf.logging.info('Sua versão do TensorFlow {}'.format(tf.__version__))
tf.logging.info('GPU disponível? {}'.format('Sim' if tf.test.is_gpu_available() else 'Não'))

%matplotlib inline

INFO:tensorflow:Recomenda-se para esta atividade uma versão >= 1.7.0
INFO:tensorflow:Sua versão do TensorFlow 1.8.0
INFO:tensorflow:GPU disponível? Não


In [2]:
!ls  # checando se a nova pasta foi criada
!ls 'dogscats/'  # checando o que tem dentro dela
DATA_DIR = 'dogscats/'  # "apontando" para nova pasta

datalab  dogscats  dogscats.zip
models	sample	test1  train  valid


In [0]:
def _img_string_to_tensor(image_string, image_size=(299, 299)):
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    # Convert from full range of uint8 to range [0,1] of float32.
    image_decoded_as_float = tf.image.convert_image_dtype(image_decoded, dtype=tf.float32)
    # Resize to expected
    image_resized = tf.image.resize_images(image_decoded_as_float, size=image_size)
    
    return image_resized

def get_dataset(file_pattern, image_size=(224, 224), shuffle=False,
                batch_size=64, num_epochs=None, buffer_size=4096):
    def _path_to_img(path):
        # Get the parent folder of this file to get it's class name
        label = tf.string_split([path], delimiter='/').values[-2]
        # Read in the image from disk
        image_string = tf.read_file(path)
        image_resized = _img_string_to_tensor(image_string, image_size)
        
        return image_resized, label

    dataset = tf.data.Dataset.list_files(file_pattern).repeat(num_epochs)

    if shuffle:
        dataset = dataset.shuffle(buffer_size)
 
    dataset = dataset.map(_path_to_img)
    dataset = dataset.batch(batch_size).prefetch(buffer_size)

    return dataset

In [4]:
H = 32
W = 32

BATCH_SIZE = 128
nasnet = hub.Module("https://tfhub.dev/google/imagenet/nasnet_large/feature_vector/1")
height, width = hub.get_expected_image_size(nasnet)

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Initialize variable module/cell_0/1x1/weights:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78faff/variables/variables with cell_0/1x1/weights
INFO:tensorflow:Initialize variable module/cell_0/beginning_bn/beta:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78faff/variables/variables with cell_0/beginning_bn/beta
INFO:tensorflow:Initialize variable module/cell_0/beginning_bn/gamma:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78faff/variables/variables with cell_0/beginning_bn/gamma
INFO:tensorflow:Initialize variable module/cell_0/beginning_bn/moving_mean:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78faff/variables/variables with cell_0/beginning_bn/moving_mean
INFO:tensorflow:Initialize variable module/cell_0/beginning_bn/moving_variance:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78f

INFO:tensorflow:Initialize variable module/cell_0/comb_iter_0/right/separable_3x3_2/pointwise_weights:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78faff/variables/variables with cell_0/comb_iter_0/right/separable_3x3_2/pointwise_weights
INFO:tensorflow:Initialize variable module/cell_0/comb_iter_1/left/bn_sep_5x5_1/beta:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78faff/variables/variables with cell_0/comb_iter_1/left/bn_sep_5x5_1/beta
INFO:tensorflow:Initialize variable module/cell_0/comb_iter_1/left/bn_sep_5x5_1/gamma:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78faff/variables/variables with cell_0/comb_iter_1/left/bn_sep_5x5_1/gamma
INFO:tensorflow:Initialize variable module/cell_0/comb_iter_1/left/bn_sep_5x5_1/moving_mean:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78faff/variables/variables with cell_0/comb_iter_1/left/bn_sep_5x5_1/moving_mean
INFO:tensorflow:Initialize var

INFO:tensorflow:Initialize variable module/cell_0/comb_iter_4/left/bn_sep_3x3_1/moving_mean:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78faff/variables/variables with cell_0/comb_iter_4/left/bn_sep_3x3_1/moving_mean
INFO:tensorflow:Initialize variable module/cell_0/comb_iter_4/left/bn_sep_3x3_1/moving_variance:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78faff/variables/variables with cell_0/comb_iter_4/left/bn_sep_3x3_1/moving_variance
INFO:tensorflow:Initialize variable module/cell_0/comb_iter_4/left/bn_sep_3x3_2/beta:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78faff/variables/variables with cell_0/comb_iter_4/left/bn_sep_3x3_2/beta
INFO:tensorflow:Initialize variable module/cell_0/comb_iter_4/left/bn_sep_3x3_2/gamma:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78faff/variables/variables with cell_0/comb_iter_4/left/bn_sep_3x3_2/gamma
INFO:tensorflow:Initialize variable module

INFO:tensorflow:Initialize variable module/cell_1/comb_iter_0/left/bn_sep_5x5_2/moving_variance:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78faff/variables/variables with cell_1/comb_iter_0/left/bn_sep_5x5_2/moving_variance
INFO:tensorflow:Initialize variable module/cell_1/comb_iter_0/left/separable_5x5_1/depthwise_weights:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78faff/variables/variables with cell_1/comb_iter_0/left/separable_5x5_1/depthwise_weights
INFO:tensorflow:Initialize variable module/cell_1/comb_iter_0/left/separable_5x5_1/pointwise_weights:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78faff/variables/variables with cell_1/comb_iter_0/left/separable_5x5_1/pointwise_weights
INFO:tensorflow:Initialize variable module/cell_1/comb_iter_0/left/separable_5x5_2/depthwise_weights:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78faff/variables/variables with cell_1/comb_iter_0/l

INFO:tensorflow:Initialize variable module/cell_1/comb_iter_1/left/separable_5x5_2/depthwise_weights:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78faff/variables/variables with cell_1/comb_iter_1/left/separable_5x5_2/depthwise_weights
INFO:tensorflow:Initialize variable module/cell_1/comb_iter_1/left/separable_5x5_2/pointwise_weights:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78faff/variables/variables with cell_1/comb_iter_1/left/separable_5x5_2/pointwise_weights
INFO:tensorflow:Initialize variable module/cell_1/comb_iter_1/right/bn_sep_3x3_1/beta:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78faff/variables/variables with cell_1/comb_iter_1/right/bn_sep_3x3_1/beta
INFO:tensorflow:Initialize variable module/cell_1/comb_iter_1/right/bn_sep_3x3_1/gamma:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78faff/variables/variables with cell_1/comb_iter_1/right/bn_sep_3x3_1/gamma
INFO:tenso

...

INFO:tensorflow:Initialize variable module/reduction_cell_0/comb_iter_0/right/separable_7x7_2/depthwise_weights:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78faff/variables/variables with reduction_cell_0/comb_iter_0/right/separable_7x7_2/depthwise_weights
INFO:tensorflow:Initialize variable module/reduction_cell_0/comb_iter_0/right/separable_7x7_2/pointwise_weights:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78faff/variables/variables with reduction_cell_0/comb_iter_0/right/separable_7x7_2/pointwise_weights
INFO:tensorflow:Initialize variable module/reduction_cell_0/comb_iter_1/right/bn_sep_7x7_1/beta:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78faff/variables/variables with reduction_cell_0/comb_iter_1/right/bn_sep_7x7_1/beta
INFO:tensorflow:Initialize variable module/reduction_cell_0/comb_iter_1/right/bn_sep_7x7_1/gamma:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78faff/varia

INFO:tensorflow:Initialize variable module/reduction_cell_0/comb_iter_4/left/bn_sep_3x3_1/beta:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78faff/variables/variables with reduction_cell_0/comb_iter_4/left/bn_sep_3x3_1/beta
INFO:tensorflow:Initialize variable module/reduction_cell_0/comb_iter_4/left/bn_sep_3x3_1/gamma:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78faff/variables/variables with reduction_cell_0/comb_iter_4/left/bn_sep_3x3_1/gamma
INFO:tensorflow:Initialize variable module/reduction_cell_0/comb_iter_4/left/bn_sep_3x3_1/moving_mean:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78faff/variables/variables with reduction_cell_0/comb_iter_4/left/bn_sep_3x3_1/moving_mean
INFO:tensorflow:Initialize variable module/reduction_cell_0/comb_iter_4/left/bn_sep_3x3_1/moving_variance:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78faff/variables/variables with reduction_cell_0/comb_ite

INFO:tensorflow:Initialize variable module/reduction_cell_1/comb_iter_0/left/bn_sep_5x5_2/gamma:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78faff/variables/variables with reduction_cell_1/comb_iter_0/left/bn_sep_5x5_2/gamma
INFO:tensorflow:Initialize variable module/reduction_cell_1/comb_iter_0/left/bn_sep_5x5_2/moving_mean:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78faff/variables/variables with reduction_cell_1/comb_iter_0/left/bn_sep_5x5_2/moving_mean
INFO:tensorflow:Initialize variable module/reduction_cell_1/comb_iter_0/left/bn_sep_5x5_2/moving_variance:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78faff/variables/variables with reduction_cell_1/comb_iter_0/left/bn_sep_5x5_2/moving_variance
INFO:tensorflow:Initialize variable module/reduction_cell_1/comb_iter_0/left/separable_5x5_1/depthwise_weights:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78faff/variables/variables wit

INFO:tensorflow:Initialize variable module/reduction_cell_1/comb_iter_1/right/bn_sep_7x7_2/moving_variance:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78faff/variables/variables with reduction_cell_1/comb_iter_1/right/bn_sep_7x7_2/moving_variance
INFO:tensorflow:Initialize variable module/reduction_cell_1/comb_iter_1/right/separable_7x7_1/depthwise_weights:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78faff/variables/variables with reduction_cell_1/comb_iter_1/right/separable_7x7_1/depthwise_weights
INFO:tensorflow:Initialize variable module/reduction_cell_1/comb_iter_1/right/separable_7x7_1/pointwise_weights:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78faff/variables/variables with reduction_cell_1/comb_iter_1/right/separable_7x7_1/pointwise_weights
INFO:tensorflow:Initialize variable module/reduction_cell_1/comb_iter_1/right/separable_7x7_2/depthwise_weights:0 from checkpoint /tmp/tfhub_modules/77e12a4b0

INFO:tensorflow:Initialize variable module/reduction_cell_1/comb_iter_4/left/separable_3x3_1/pointwise_weights:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78faff/variables/variables with reduction_cell_1/comb_iter_4/left/separable_3x3_1/pointwise_weights
INFO:tensorflow:Initialize variable module/reduction_cell_1/comb_iter_4/left/separable_3x3_2/depthwise_weights:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78faff/variables/variables with reduction_cell_1/comb_iter_4/left/separable_3x3_2/depthwise_weights
INFO:tensorflow:Initialize variable module/reduction_cell_1/comb_iter_4/left/separable_3x3_2/pointwise_weights:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78faff/variables/variables with reduction_cell_1/comb_iter_4/left/separable_3x3_2/pointwise_weights
INFO:tensorflow:Initialize variable module/reduction_cell_1/prev_1x1/weights:0 from checkpoint /tmp/tfhub_modules/77e12a4b00ee2743f22a9dd90b9d66aa8f78faff

In [5]:
print("Height:", height, "Width:", width)

Height: 331 Width: 331


In [0]:
train_files = os.path.join(DATA_DIR, 'train', '**/*.8.jpg')
test_files = os.path.join(DATA_DIR, 'valid', '**/*8.jpg')

num_epochs = 5

# keep results for plotting
train_loss_results = []
train_accuracy_results = []

train_ds = get_dataset(train_files, batch_size=BATCH_SIZE, shuffle=True, image_size=(height, width), num_epochs=10)
test_ds = get_dataset(test_files, batch_size=BATCH_SIZE, image_size=(height, width), num_epochs=1)

In [0]:
# create a iterator of the correct shape and type
iter = tf.data.Iterator.from_structure(train_ds.output_types, train_ds.output_shapes)
x, y = iter.get_next()


# create the initialisation operations
train_init_op = iter.make_initializer(train_ds)
test_init_op = iter.make_initializer(test_ds)

In [0]:
logits = nasnet(x)

In [0]:
dense = tf.layers.dense(logits, 64, activation=tf.tanh)
y_ = tf.layers.dense(dense, 2)

mapping_string = tf.constant(["cats", "dogs"])
table = tf.contrib.lookup.index_table_from_tensor(
    mapping=mapping_string, num_oov_buckets=1, default_value=-1)

ids = table.lookup(y)
loss = tf.losses.sparse_softmax_cross_entropy(labels=ids, logits=y_)
train_op = tf.train.AdamOptimizer().minimize(loss)

prediction = tf.argmax(y_, axis=1, output_type=tf.int32)
accuracy = tf.metrics.accuracy(labels=ids, predictions=prediction)

In [15]:
with tf.Session() as sess:
    sess.run(train_init_op) # switch to train dataset
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    sess.run(tf.tables_initializer())
    _, loss_value, acc = sess.run([train_op, loss, accuracy])
    print(loss_value, acc)
    sess.run(test_init_op) # switch to val dataset
    print(sess.run(accuracy))

1.3158066 (0.0, 0.0)
(0.0, 0.5202703)
